In [75]:
import sys
import subprocess
import os
import glob

import pandas as pd

In [71]:
# ARGS
GWAS = "/scratch/tmp-magma_gwas/BMI_Yengo2018.txt.10UP.1.5DOWN.genes.raw"
WGCNA_MODULE_GENELIST = "/raid5/projects/timshel/sc-genetics/sc-genetics/out/out.wgcna/nn_lira_sema/tables/nn_lira_sema_per_brain_area_run1_cell_cluster_module_genes.csv"
OUTFILES_PREFIX = "magma_test_run_bmi"
# DIR_WGCNA_MODULES = "/raid5/projects/timshel/sc-genetics/sc-genetics/out/out.wgcna/nn_lira_sema/tables"
# PREFIX_WGCNA_MODULES = "nn_lira_sema_per_brain_area_run1"

In [50]:
def map_ensembl_genes_mouse_to_human(df):
    """
    DESCRIPTION
        function to map mouse ensembl genes to human ortholog genes.
    INPUT
       df: a dataframe with two columns: "annotation" and "gene". "gene" column should contain Ensembl mouse gene IDs to be mapped.
    OUTPUT
       df: input dataframe with mapped genes. Mouse genes that could not be mapped are removed.
       file_mapping_summary: a summary file with mapping stats

    REMARKS
        We assume the file_mapping contains only 1-1 mapping (which is true for gene_annotation.hsapiens_mmusculus_unique_orthologs.GRCh37.ens_v91.txt.gz).
        Otherwise the .map() function might fail
    """
    file_out_mapping_stats = "{}.ortholog_mapping_stats.txt".format(OUTFILES_PREFIX)
    
    file_mapping = "/raid5/projects/timshel/sc-genetics/sc-genetics/data/gene_annotations/gene_annotation.hsapiens_mmusculus_unique_orthologs.GRCh37.ens_v91.txt.gz"
    ### SNIPPET
    # ensembl_gene_id chromosome_name start_position  end_position    mmusculus_homolog_ensembl_gene  mmusculus_homolog_orthology_confidence
    # ENSG00000138593 15      49280673        49338760        ENSMUSG00000035093      1
    # ENSG00000166351 21      14982498        15013906        ENSMUSG00000095294      0
    # ENSG00000168675 18      13217497        13652754        ENSMUSG00000024544      1
    
    df_mapping = pd.read_csv(file_mapping, delim_whitespace=True, usecols=["ensembl_gene_id", "mmusculus_homolog_ensembl_gene"], index_col=1) # index is MOUSE genes
    
    ### map ortholog genes
    df["gene_human"] = df["gene"].map(df_mapping["ensembl_gene_id"]) # df["gene"] is mouse genes. df_mapping["ensembl_gene_id"] is human.
    # ^ .map() returns NaN values for genes not mapped. 

    ### make summary of mapping
    df_summary = df.groupby("annotation")["gene_human"].agg({'n_genes_input': lambda x: len(x),
                                                         'n_genes_output': lambda x: len(x)-sum(pd.isnull(x)), 
                                                        'n_genes_not_mapped' : lambda x: sum(pd.isnull(x)),
                                                        'pct_genes_not_mapped': lambda x: "{:.2f}".format(sum(pd.isnull(x))/float(len(x))*100)})
    # ^ we use pd.isnull() instead of np.isnan() because of the issue described here: https://stackoverflow.com/a/36001191/6639640
    df_summary.sort_values(by=['n_genes_output'], inplace=True) 
    df_summary.to_csv(file_out_mapping_stats, sep="\t")
    
    ### final processing
    df.dropna(axis=0, inplace=True) # remove non-mapped genes
    df['gene'] = df['gene_human'] # replace mouse genes with human genes
    df.drop(['gene_human'], axis=1, inplace=True) # remove human gene column
    
    return df


def map_hs_ensembl_to_entrez(df):
    # Entrezgene to ensemb_gene_id mapping file path for MAGMA
    file_mapping_hs_ensembl2entrez = "/projects/tp/tmp-bmi-brain/data/mapping/gene_annotation_hsapiens.txt.gz"
    df_mapping_ensembl2entrez = pd.read_csv(file_mapping_hs_ensembl2entrez, sep="\t") # shape=(64719, 3)
    df_mapping_ensembl2entrez.drop_duplicates('ensembl_gene_id', keep='first', inplace=True) # shape=(63967, 3)
    df_mapping_ensembl2entrez.drop_duplicates('entrezgene', keep='first', inplace=True) # shape=(25233, 3)
    df_mapping_ensembl2entrez.set_index("ensembl_gene_id", inplace=True) # set index so '.map()' will work
    
    ### map
    df["gene_entrez"] = df["gene"].map(df_mapping_ensembl2entrez["entrezgene"])
    # ^ .map() returns NaN values for genes not mapped. 
    
    ### summarise mapping stats (optional)
    df_summary = df.groupby("annotation")["gene_entrez"].agg({'n_genes_input': lambda x: len(x),
                                                         'n_genes_output': lambda x: len(x)-sum(pd.isnull(x)), 
                                                        'n_genes_not_mapped' : lambda x: sum(pd.isnull(x)),
                                                        'pct_genes_not_mapped': lambda x: "{:.2f}".format(sum(pd.isnull(x))/float(len(x))*100)})
    # ^ we use pd.isnull() instead of np.isnan() because of the issue described here: https://stackoverflow.com/a/36001191/6639640
    df_summary.sort_values(by=['n_genes_output'], inplace=True)
    # print(df_summary)
    
    ### final processing
    df.dropna(axis=0, inplace=True) # remove non-mapped genes
    df['gene_entrez'] = df['gene_entrez'].astype(int) # convert to int (otherwise it will be float). But conversion cannot be done before NaNs have been removed
    # df['gene'] = df['gene_entrez'] # replace mouse genes with human genes
    # df.drop(['gene_entrez'], axis=1, inplace=True) # remove human gene column
    
    return(df)


In [67]:
df_multi_gene_set = pd.read_csv(WGCNA_MODULE_GENELIST, usecols=["module", "ensembl"]) 
# ^ 'module' = first column; will later be renamed to 'annotation'
# ^ 'ensembl' = second column; will later be renamed to 'gene'
df_multi_gene_set.columns = ["annotation", "gene"] # setting or renaming column names
### map to human
df_multi_gene_set_hs = map_ensembl_genes_mouse_to_human(df_multi_gene_set)
df_multi_gene_set_hs.head()
### map ensembl to entrez
df_multi_gene_set_hs_entrez = map_hs_ensembl_to_entrez(df_multi_gene_set)
df_multi_gene_set_hs_entrez.head()

/tools/anaconda/3-4.4.0/envs/py3_anaconda3_PT180510/lib/python3.6/site-packages/ipykernel/__main__.py:34: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
/tools/anaconda/3-4.4.0/envs/py3_anaconda3_PT180510/lib/python3.6/site-packages/ipykernel/__main__.py:63: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,annotation,gene,gene_entrez
0,darkkhaki,ENSG00000197971,4155
1,darkkhaki,ENSG00000123560,5354
2,darkkhaki,ENSG00000105695,4099
3,darkkhaki,ENSG00000168314,4336
4,darkkhaki,ENSG00000205116,643965


In [81]:
## write output file
fileout_magma_geneset = "{}.set_annot.txt".format(OUTFILES_PREFIX)
df_multi_gene_set_hs_entrez.drop("gene", axis='columns').to_csv(fileout_magma_geneset, sep="\t", header=False, index=False) # no header, no index

In [83]:
# magma --gene-results <GWAS>.genes.raw --set-annot <INPUT> --out <GWAS.MODULE>
cmd = "/tools/magma/1.06/magma --gene-results {file_gwas_genes_raw} --set-annot {file_geneset} set-col=1 gene-col=2 --out {fileout_prefix}".format(file_gwas_genes_raw=GWAS, file_geneset=fileout_magma_geneset, fileout_prefix=OUTFILES_PREFIX)
cmd

'/tools/magma/1.06/magma --gene-results /scratch/tmp-magma_gwas/BMI_Yengo2018.txt.10UP.1.5DOWN.genes.raw --set-annot magma_test_run_bmi.set_annot.txt set-col=1 gene-col=2 --out magma_test_run_bmi'

In [84]:
subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)

b'Welcome to MAGMA v1.06 (linux)\nUsing flags:\n\t--gene-results /scratch/tmp-magma_gwas/BMI_Yengo2018.txt.10UP.1.5DOWN.genes.raw\n\t--set-annot magma_test_run_bmi.set_annot.txt set-col=1 gene-col=2\n\t--out magma_test_run_bmi\n\nStart time is 09:30:31, Sunday 30 Sep 2018\n\nReading file /scratch/tmp-magma_gwas/BMI_Yengo2018.txt.10UP.1.5DOWN.genes.raw... \n\t17625 genes read from file\nLoading gene-set annotation...\nReading file magma_test_run_bmi.set_annot.txt... \n\tdetected 2 variables in file\n\tusing variable: variable 2 (gene ID)\n\tusing variable: variable 1 (set ID)\n\tfound 33 gene sets containing genes defined in genotype data (containing a total of 5116 unique genes)\n\nStarting gene-level analysis...\n\tconditioning competitive gene-set analysis and gene covariate analysis on variables:\t\n\t\tgenesize, log_genesize (internal)\n\t\tgenedensity, log_genedensity (internal)\n\t\tsamplesize, log_samplesize (internal)\n\t\tinverse_mac, log_inverse_mac (internal)\n\tprocessing b